# Майнор "Прикладные задачи анализа данных"
## Предсказание цены акции по экономическим новостям

---
### Задания:
1. Предварительная обработка текстов и эксплоративный анализ
2. Baseline алгоритм
3. Творческая часть

Входные данные:
* Новости о компании "Газпром", начиная с 2010 года
* Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
    * цена открытия (Open)
    * цена закрытия (ClosingPrice)
    * максимальная цена за день (DailyHigh)
    * минимальная цена за день (DailyLow) 
    * объем бумаг (VolumePcs)

---

In [1]:
import numpy as np
import re
import pymorphy2
import nltk
#remove comment to download
#nltk.download('stopwords')
from nltk.corpus import stopwords
from scipy.stats.stats import pearsonr
import pandas as pd

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB


Функция для загрузки таблиц. Также нам пригодится список дат, которые есть в обоих таблицах.

In [8]:
def load_data():
    txts = pd.read_csv('data/texts.csv', parse_dates=[0])
    prices = pd.read_csv('data/gazprom_prices.csv', sep=';', parse_dates=[0])
    prices.sort_values('Date', inplace=True)
    txts.sort_values('date', inplace=True)
    prices.set_index('Date', inplace=True)
    prices['ClosingPrice'] = pd.to_numeric(prices['ClosingPrice'].str.replace(",","."))
    txts.set_index('date', inplace=True)
    news_idx = txts.index.unique()
    pr_idx = prices.index.unique()
    idx = list(set(pr_idx).intersection(news_idx))
    
    return txts, prices, idx

## Часть 1. Вводная [3 балла]

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. Ответьте на следующие вопросы:
* Есть ли корреляция между средней длинной текста за день и ценой закрытия?
* Есть ли корреляция между количеством упоминаний Алексея Миллера  и ценой закрытия? Учтите разные варианты написания имени.
* Упоминаний какого газопровода в статьях больше: 
    * "северный поток"
    * "турецкий поток"?
* Кого упоминают чаще:
    * Алексея Миллера
    * Владимира Путина?
* О каких санкциях пишут в статьях?

In [9]:
df, pr_all, idx = load_data()

In [10]:
df.head(4)

,text
date,
2010-01-02,"""Газпром"" не исключает в 2010 г. выпуска обли..."
2010-01-19,"""Газпром"" готов забирать весь объем азербайдж..."
2010-01-28,"Консорциум во главе с российским ОАО ""Газпром..."
2010-02-07,Газпромбанк открыл на Кипре дочернюю компанию...


In [11]:
pr_all.head(4)

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
Date,,,,,
2010-01-02,"184,74000",189.85,"190,40000","183,50000",76298175
2010-01-03,"168,70000",168.20,"170,71000","166,33000",58570262
2010-01-04,"172,49000",175.00,"176,14000","172,33000",94994135
2010-01-06,"158,20000",159.26,"160,31000","154,39000",67031024


In [12]:
# токенизация

i = 0
j = 0
all_text = []
prog = re.compile('[а-яё-]+')
for i in range(len(df)):
    s = df['text'][i]
    df['text'].iloc[i] = prog.findall(s.lower())
    for j in range(len(df['text'][i])):
        all_text.append(df['text'][i][j])

In [13]:
# функция создания всех вариаций падежей для конкретного слова

case = ['nomn', 'gent', 'datv', 'accs', 'ablt', 'loct']
def cases(word):
    changed_word = []
    morph = pymorphy2.MorphAnalyzer()
    parsed_word = morph.parse(word)[0]
    for i in range(len(case)):
        changed_word.append(parsed_word.inflect({case[i]}).word)
    return changed_word

In [14]:
# корреляция длины текста и цены закрытия
# корреляция длины Александра Миллера и цены закрытия

lens = []
prices = []
countes = []
cases_miller = cases('миллер')
        
texts = df[df.index.isin(idx)]
prs = pr_all[pr_all.index.isin(idx)]

In [15]:
for i in range(len(idx)):
    lens.append(len(texts.loc[texts.index[i],'text']))
    temp = prs.loc[prs.index[i],'ClosingPrice']
    prices.append(temp) 
    count = 0
    freqDist = nltk.FreqDist(texts.loc[texts.index[i], 'text'])
    for j in range(len(cases_miller)):
        count += freqDist[cases_miller[j]]
    countes.append(count)

In [16]:
print(pearsonr(lens, prices))
print(pearsonr(countes, prices))

(0.012405743888311147, 0.67309599859248292)
(0.0055160956175063303, 0.8511986601879219)


Коэффициент корреляции для связи длинны текста с ценой закрытия равен 0.012

Коэффициент корреляции для связи Александра Миллера с ценой закрытия равен 0.005

Как мы видим - корреляция очень маленькая => связь между величинами не берем в расчет

In [17]:
# Сравнение потоков и сравнение Путина и Миллера

cases_nord = cases('северный')
cases_turkey = cases('турецкий')
cases_putin = cases('путин')
cases_miller = cases('миллер')

freqDist = nltk.FreqDist(all_text)

count_nord = 0
count_turkey = 0
count_putin = 0
count_miller = 0

for j in range(len(cases_nord)):
    count_nord += freqDist[cases_nord[j]]
    count_turkey += freqDist[cases_turkey[j]]
    count_putin += freqDist[cases_putin[j]]
    count_miller += freqDist[cases_miller[j]]
    
print('Турецкий поток:', count_turkey, "\nСеверный поток:", count_nord)
print('Владимир Путин:', count_putin, "\nАлександр Миллер:", count_miller)

Турецкий поток: 61 
Северный поток: 36
Владимир Путин: 101 
Александр Миллер: 176


In [18]:
# Санкции

df2 = pd.read_csv('data/texts.csv')
count = 0
cases_sanction = cases('санкция')
sanctions = []

for i in range(len(df['text'])):
    for j in range(len(cases_sanction)):
        index = df2['text'][i].find(cases_sanction[j])
        
        new = ''
        if index != -1:
            index -= 2
            while (df2['text'][i][index] != ' '):
                new += df2['text'][i][index]
                index -= 1
                new1 = new[::-1]
            sanctions.append(new1)
            break

In [19]:
sanctions

['банкам',
 'под',
 'осложняют',
 'под',
 'и',
 'западные',
 'экономическими',
 'антироссийскими',
 'персональные',
 'на',
 'затронутых',
 'штрафные',
 'числе',
 'что',
 'под',
 'западные',
 'запрета,',
 'под',
 'финансовые',
 'ввести',
 'ждут',
 'международные']

Видим, что санкции могут быть:
    Западные (x2),
    Экономические (x1),
    Антироссийские (x1),
    Персональные (x1),
    Штрафные (х1),
    Затронутые (x1),
    Финансовые (x1),
    Международные (x1).
    
Чаше всего пишут про западные санкции.

(2 метод выполнения этого пункта представлен ниже)

In [20]:
from nltk.text import Text
textList = Text(all_text)
textList.concordance('санкции')

Displaying 22 of 22 matches:
тране подпадающей под международные санкции на кубе россия которая не хочет тер
тране подпадающей под международные санкции на кубе газпром-медиа нашел покупат
адельца геннадия тимченко попал под санкции сша может в будущем выйти из черног
о уже несколько лет пытается ввести санкции к компаниям и их топ-менеджерам за 
т подпасть под визовые и финансовые санкции которые европа планирует ввести в о
енеджеров но после анализа ситуации санкции могут распространиться и на деятель
дному из проектов сланцевой нефти с санкции могут затронуть и другое сп общей т
дному из проектов сланцевой нефти с санкции могут затронуть и другое сп общей т
ржит госбанки попавшие под западные санкции в список банков которые получат воз
мотря на отсутствие прямого запрета санкции могут привести к трудностям с поста
транить наши опасения компанию ждут санкции пока еврокомиссия получила хорошие 
ашим финансовым институтам развития санкции - объяснил министр ведет переговоры
порте газпр

## Часть 2. Классификационная [3 балла]
Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия.
Для этого:
* бинаризуйте признак "цена закрытия":  новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случаея;
* составьте бучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем: 
* ClosingPrice_bin – бинарый целевой признак
* слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части. 

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации.
Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

---

Загрузим данные в датафреймы

In [21]:
txts, prices, idx = load_data()
txts.head(5)

,text
date,
2010-01-02,"""Газпром"" не исключает в 2010 г. выпуска обли..."
2010-01-19,"""Газпром"" готов забирать весь объем азербайдж..."
2010-01-28,"Консорциум во главе с российским ОАО ""Газпром..."
2010-02-07,Газпромбанк открыл на Кипре дочернюю компанию...
2010-02-09,"""Газпром"" вновь понизил прогноз экспорта в Ев..."


In [22]:
prices.head(4)

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs
Date,,,,,
2010-01-02,"184,74000",189.85,"190,40000","183,50000",76298175
2010-01-03,"168,70000",168.20,"170,71000","166,33000",58570262
2010-01-04,"172,49000",175.00,"176,14000","172,33000",94994135
2010-01-06,"158,20000",159.26,"160,31000","154,39000",67031024


In [23]:
print(len(txts))
print(len(prices))
print(txts.dtypes)
print(prices.dtypes)

1203
1988
text    object
dtype: object
Open             object
ClosingPrice    float64
DailyHigh        object
DailyLow         object
VolumePcs         int64
dtype: object


In [24]:
print(txts.isnull().any())
print(prices.isnull().any())

text    False
dtype: bool
Open             True
ClosingPrice    False
DailyHigh        True
DailyLow         True
VolumePcs       False
dtype: bool


Получим вектор бинарных значений

In [25]:
binarized = []
for i in np.arange(0,len(prices)-1):
    if prices['ClosingPrice'].iloc[i+1] >= prices['ClosingPrice'].iloc[i]:
        binarized.append(1)
    else:
        binarized.append(0)

binarized.append(np.nan) #т.к. мы не знаем, как закрылись торги в день, следующий за последним в таблице
prices['ClosingPrice_bin'] = binarized

In [26]:
df_prices = prices.drop(['Open', 'DailyHigh', 'DailyLow', 'VolumePcs'], axis=1)
df_prices.head(10)

,ClosingPrice,ClosingPrice_bin
Date,,
2010-01-02,189.85,0.0
2010-01-03,168.20,1.0
2010-01-04,175.00,0.0
2010-01-06,159.26,0.0
2010-01-07,143.20,1.0
2010-01-09,161.79,0.0
2010-01-10,160.54,1.0
2010-01-11,169.89,1.0
2010-01-12,182.49,1.0


Найдем пересечения между множеством дат новостей и аналогичным множеством дат торгов

In [28]:
df_prices = df_prices[df_prices.index.isin(idx)]
txts = txts[txts.index.isin(idx)]
print(len(df_prices))
print(len(texts))

1159
1159


In [29]:
txts['ClosingPrice_bin'] = df_prices['ClosingPrice_bin'].values
txts.head(5)

,text,ClosingPrice_bin
date,,
2010-01-02,"""Газпром"" не исключает в 2010 г. выпуска обли...",0.0
2010-01-19,"""Газпром"" готов забирать весь объем азербайдж...",0.0
2010-01-28,"Консорциум во главе с российским ОАО ""Газпром...",1.0
2010-02-07,Газпромбанк открыл на Кипре дочернюю компанию...,1.0
2010-02-09,"""Газпром"" вновь понизил прогноз экспорта в Ев...",1.0


In [30]:
txts.isnull().any()

text                False
ClosingPrice_bin    False
dtype: bool

In [31]:
X_train = txts[txts.index < pd.Timestamp(2016,1,1)]

In [32]:
X_test = txts[txts.index >= pd.Timestamp(2016,1,1)]

Создадим два вектора - с и без применения tf-idf, чтобы проверить, дает ли улучшения использование tf-idf

In [33]:
count_vect = CountVectorizer()
tfdif_vect = TfidfVectorizer(lowercase=True, stop_words=None, use_idf=True, ngram_range=(1,1), smooth_idf=False)                        
vector_model = tfdif_vect.fit(X_train['text'])
tfidf_vector = vector_model.transform(X_train['text'])
count_vector = count_vect.fit_transform(X_train['text'])

In [34]:
x_train = tfidf_vector.toarray()
x_train_c = count_vector.toarray()

In [35]:
y_train = X_train['ClosingPrice_bin'].values

In [36]:
tfidf_vector_test = vector_model.transform(X_test['text'])
count_vector_test = count_vect.transform(X_test['text'])

In [37]:
x_test = tfidf_vector_test.toarray()
x_test_c = count_vector_test.toarray()

In [38]:
y_test = X_test['ClosingPrice_bin'].values

Построим модель классификации. В качестве нее был выбран мультиномиальный наивный Байес.

In [39]:
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
predictions = clf.predict(x_test)

In [41]:
accuracy_score(y_test, predictions)

0.58139534883720934

In [42]:
f1_score(y_test, predictions)

0.23943661971830987

In [43]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

        0.0       0.60      0.88      0.71       151
        1.0       0.49      0.16      0.24       107

avg / total       0.55      0.58      0.52       258



Проверим, дает ли улучшения использование tf-idf (ниже модель обучается на векторе, полученном без использования tf-idf)

In [44]:
clf_c = MultinomialNB()
clf_c.fit(x_train_c, y_train)
predictions_c = clf_c.predict(x_test_c)
accuracy_score(y_test, predictions_c)

0.55426356589147285

Как видим, без tf-idf accuracy_score несколько ниже

Попробуем разные параметры альфа в модели наивного Байеса

In [45]:
alphas = [0.0001, 0.001, 0.01, 0.1, 1]
for a in alphas:
    cl = MultinomialNB(alpha=a)
    cl.fit(x_train, y_train)
    preds = cl.predict(x_test)
    print('Alpha: {} - accuracy: {}'.format(a,accuracy_score(y_test, preds)))

Alpha: 0.0001 - accuracy: 0.5697674418604651
Alpha: 0.001 - accuracy: 0.5813953488372093
Alpha: 0.01 - accuracy: 0.5930232558139535
Alpha: 0.1 - accuracy: 0.5697674418604651
Alpha: 1 - accuracy: 0.5813953488372093


## Часть 3. Творческая [4 балла]
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. 
Направления развития:
* Морфологический признаки: 
    * использовать в качестве признаков только существительные или только именованные сущности;
* Модели скрытых тем:
    * использовать в качестве признаков скрытые темы;
    * использовать в качестве признаков динамические скрытые темы 
    пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
* Синтаксические признаки:
    * использовать SOV-тройки в качестве признаков
    * кластеризовать SOV-тройки по усредненным эмбеддингам  (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/ или https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) и использовать только центроиды кластеров в качестве признаков
* что-нибудь еще     

Для начала попробуем использовать другие модели для классификации:

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model

In [47]:
knn = KNeighborsClassifier(n_neighbors=5)
logregr = linear_model.LogisticRegression(penalty="l2", fit_intercept=True, max_iter=100, C=1, solver="lbfgs", random_state=123)

In [48]:
logregr.fit(x_train, y_train)
logregr.score(x_test, y_test)

0.55038759689922478

In [49]:
knn.fit(x_train, y_train)
knn.score(x_test, y_test)

0.50387596899224807

К сожалению, улучшений это не дало, наоборот - результат стал хуже. Попробуем использовать существительные и именованные сущности в качестве признаков.
Для извлечения именованных сущностей используем библиотеку natasha *(можно установить командой pip install natasha)*

In [52]:
import sys
!{sys.executable} -m pip install natasha

  Using cached natasha-0.9.0-py2.py3-none-any.whl
  Using cached yargy-0.10.0-py2.py3-none-any.whl
  Using cached backports.functools_lru_cache-1.3-py2.py3-none-any.whl
  Using cached intervaltree-2.1.0.tar.gz
  Running setup.py bdist_wheel for intervaltree ... done
  Stored in directory: /Users/miron/Library/Caches/pip/wheels/89/40/01/fa05b5a8202a472fb143815e7589fdf74369e710ca675cad11
Successfully built intervaltree
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [58]:
from natasha import NamesExtractor

Извлечем все существительные (код взят из примера отсюда: https://stackoverflow.com/questions/33587667/extracting-all-nouns-from-a-text-file-using-nltk)

In [63]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/miron/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Можем извлечь все существительные из всех текстов.

In [64]:
# function to test if something is a noun
is_noun = lambda pos: pos[:2] == 'NN'
# do the nlp stuff
tokenized = nltk.word_tokenize(str(all_text))
nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]

In [131]:
import string
from nltk.corpus import stopwords
 
def get_nouns(file_text):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = nltk.word_tokenize(file_text)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    nouns = [i for i in nouns if ( i not in string.punctuation )]
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])
    nouns = [i for i in nouns if ( i not in stop_words )]
    nouns = [i.replace("«", "").replace("»", "") for i in nouns]
 
    return ' '.join(nouns)

Но мы возьмем существительные из обучающих и тестовых данных отдельно:

In [132]:
nouns_test = []
nouns_train = []

for val in X_train['text'].values:
    nouns_train.append(get_nouns(val))

In [133]:
for val in X_test['text'].values:
    nouns_test.append(get_nouns(val))

С русским nltk работает не так хорошо пока что, но все равно попробуем обучить нашу модель

In [113]:
tfdif_vect = TfidfVectorizer(lowercase=True, stop_words=None, use_idf=True, ngram_range=(1,1), smooth_idf=False)                        
vector_model = tfdif_vect.fit(nouns_train)
tfidf_vector = vector_model.transform(nouns_train)

In [114]:
count_vector = count_vect.fit_transform(nouns_train)
x_train = tfidf_vector.toarray()
x_train_c = count_vector.toarray()

In [115]:
tfidf_vector_test = vector_model.transform(nouns_test)
count_vector_test = count_vect.transform(nouns_test)
x_test = tfidf_vector_test.toarray()
x_test_c = count_vector_test.toarray()

In [116]:
cl = MultinomialNB(alpha=0.01)

In [117]:
cl.fit(x_train, y_train)
preds = cl.predict(x_test)
print('accuracy: {}'.format(accuracy_score(y_test, preds)))

accuracy: 0.5736434108527132


In [118]:
cl.fit(x_train_c, y_train)
preds_c = cl.predict(x_test_c)
print('accuracy: {}'.format(accuracy_score(y_test, preds_c)))

accuracy: 0.5775193798449613


In [119]:
extractor = NamesExtractor()

In [120]:
names_train = []
names_test = []

In [121]:
def extract_names(text):
    matches = extractor(text)
    m = []
    for match in matches:
        start, stop = match.span
        m.append(text[start:stop])
    return ' '.join(m)

In [122]:
for val in X_train['text'].values:
    names_train.append(extract_names(val))

In [123]:
for val in X_test['text'].values:
    names_test.append(extract_names(val))

In [124]:
names_train[1]

'Алексей Миллер Миллер'

In [125]:
tfdif_vect = TfidfVectorizer(lowercase=True, stop_words=None, use_idf=True, ngram_range=(1,1), smooth_idf=False)                        
vector_model = tfdif_vect.fit(names_train)
tfidf_vector = vector_model.transform(names_train)
count_vector = count_vect.fit_transform(names_train)
x_train = tfidf_vector.toarray()
x_train_c = count_vector.toarray()
tfidf_vector_test = vector_model.transform(names_test)
count_vector_test = count_vect.transform(names_test)
x_test = tfidf_vector_test.toarray()
x_test_c = count_vector_test.toarray()
cl = MultinomialNB(alpha=0.01)
cl.fit(x_train, y_train)
preds = cl.predict(x_test)
print('accuracy: {}'.format(accuracy_score(y_test, preds)))
cl.fit(x_train_c, y_train)
preds_c = cl.predict(x_test_c)
print('accuracy: {}'.format(accuracy_score(y_test, preds_c)))

accuracy: 0.5658914728682171
accuracy: 0.5775193798449613


In [136]:
tf_vect = TfidfVectorizer(lowercase=True, stop_words=None, use_idf=False, ngram_range=(1,1), smooth_idf=False)
vector_model = tf_vect.fit(nouns_train)
tf_vector = vector_model.transform(nouns_train)
x_train = tf_vector.toarray()
tf_vector_test = vector_model.transform(nouns_test)
x_test = tf_vector_test.toarray()
cl_1 = MultinomialNB(alpha=0.01)
cl_1.fit(x_train, y_train)
preds = cl_1.predict(x_test)
print('accuracy: {}'.format(accuracy_score(y_test, preds)))

accuracy: 0.5697674418604651
